In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
n_players = 5
n_spins = 100
starting_wealth = 1000
bet_size = 100

# European roulette has 37 numbers (0-36)
# Betting on red/black gives 18/37 chance of winning
p_win = 18/37
payout = 2  # 1:1 payout for red/black bets

# Generate wealth paths for players
player_wealths = np.zeros((n_players, n_spins + 1))
player_wealths[:, 0] = starting_wealth

# Generate casino wealth path
casino_wealth = np.zeros(n_spins + 1)
casino_wealth[0] = 0

# Simulate spins
for i in range(n_spins):
    results = np.random.random(n_players) < p_win
    wins = results * bet_size * (payout - 1)
    losses = ~results * bet_size
    net_results = wins - losses
    
    player_wealths[:, i+1] = player_wealths[:, i] + net_results
    casino_wealth[i+1] = casino_wealth[i] - sum(net_results)

# Find first zero for each player and set all future values to zero
for i in range(n_players):
    zero_indices = np.where(player_wealths[i] <= 0)[0]
    if len(zero_indices) > 0:
        first_zero = zero_indices[0]
        player_wealths[i, first_zero:] = 0

# Create figure with subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Player Wealth Paths', 'Casino Wealth Path')
)

# Define colors
colors = ['rgba(255, 0, 255, 1)', 'rgba(0, 255, 255, 1)', 
         'rgba(0, 255, 0, 1)', 'rgba(255, 165, 0, 1)', 
         'rgba(255, 0, 0, 1)']

# Create frames for animation
frames = []
for step in range(n_spins + 1):
    frame = {"data": [], "name": str(step)}
    
    # Calculate axis limits for current step
    y_min_players = min(np.min(player_wealths[:, :step+1]), 0)
    y_max_players = max(np.max(player_wealths[:, :step+1]), starting_wealth)
    y_min_casino = min(np.min(casino_wealth[:step+1]), 0)
    y_max_casino = max(np.max(casino_wealth[:step+1]), 0)
    
    # Add player traces
    for i in range(n_players):
        frame["data"].append(
            go.Scatter(
                x=np.arange(step + 1),
                y=player_wealths[i, :step + 1],
                mode='lines',
                line=dict(color=colors[i % len(colors)], width=2),
                name=f'Player {i+1}'
            )
        )
    
    # Add casino trace
    frame["data"].append(
        go.Scatter(
            x=np.arange(step + 1),
            y=casino_wealth[:step + 1],
            mode='lines',
            line=dict(color='rgba(255, 215, 0, 1)', width=2),
            name='Casino'
        )
    )
    
    # Update axis ranges for this frame
    frame["layout"] = {
        "xaxis": {"range": [0, max(step + 1, 5)]},
        "xaxis2": {"range": [0, max(step + 1, 5)]},
        "yaxis": {"range": [y_min_players * 1.1, y_max_players * 1.1]},
        "yaxis2": {"range": [y_min_casino * 1.1, y_max_casino * 1.1]}
    }
    
    frames.append(frame)

# Add initial traces (empty)
for i in range(n_players):
    fig.add_trace(
        go.Scatter(
            x=[0],
            y=[starting_wealth],
            mode='lines',
            line=dict(color=colors[i % len(colors)], width=2),
            name=f'Player {i+1}'
        ),
        row=1, col=1
    )

# Add initial casino trace (empty)
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        mode='lines',
        line=dict(color='rgba(255, 215, 0, 1)', width=2),
        name='Casino'
    ),
    row=1, col=2
)

# Update layout with animation settings
fig.update_layout(
    height=500,
    width=1200,
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white'),
    updatemenus=[{
        'type': 'buttons',
        'showactive': False,
        'buttons': [{
            'label': 'Play',
            'method': 'animate',
            'args': [None, {
                'frame': {'duration': 50, 'redraw': True},
                'fromcurrent': True,
                'transition': {'duration': 0}
            }]
        }, {
            'label': 'Pause',
            'method': 'animate',
            'args': [[None], {
                'frame': {'duration': 0, 'redraw': False},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }]
        }]
    }],
    sliders=[{
        'currentvalue': {'prefix': 'Spin: '},
        'pad': {'t': 50},
        'len': 0.9,
        'x': 0.1,
        'steps': [{
            'args': [[str(i)], {
                'frame': {'duration': 0, 'redraw': True},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }],
            'label': str(i),
            'method': 'animate'
        } for i in range(n_spins + 1)]
    }]
)

# Update axes
for i in range(1, 3):
    fig.update_xaxes(
        title='Number of Spins',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )
    fig.update_yaxes(
        title='Wealth ($)' if i == 1 else 'Casino Profit ($)',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )

# Set initial axis ranges
fig.update_xaxes(range=[0, 5], row=1, col=1)
fig.update_xaxes(range=[0, 5], row=1, col=2)
fig.update_yaxes(range=[0, starting_wealth * 1.1], row=1, col=1)
fig.update_yaxes(range=[0, starting_wealth * 1.1], row=1, col=2)

# Add frames to figure
fig.frames = frames

fig.show()


# Set random seed for reproducibility 
np.random.seed(42)  # Chosen to demonstrate a "lucky" sequence

# Simulation parameters for lucky player example
n_spins = 50
starting_wealth = 1000
bet_size = 100
p_win = 18/37  # European roulette probability
payout = 2     # 1:1 payout

# Initialize wealth paths
player_wealth = np.zeros(n_spins + 1)
casino_wealth = np.zeros(n_spins + 1)
player_wealth[0] = starting_wealth

# Simulate spins with a "lucky" sequence
for i in range(n_spins):
    result = np.random.random() < p_win
    win = result * bet_size * (payout - 1)
    loss = (not result) * bet_size
    net_result = win - loss
    
    player_wealth[i+1] = player_wealth[i] + net_result
    casino_wealth[i+1] = casino_wealth[i] - net_result

# Create figure
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Lucky Player Wealth Path', 'Casino Loss Path')
)

# Add player trace
fig.add_trace(
    go.Scatter(
        x=np.arange(n_spins + 1),
        y=player_wealth,
        mode='lines',
        line=dict(color='rgba(0, 255, 0, 1)', width=2),
        name='Lucky Player'
    ),
    row=1, col=1
)

# Add casino trace
fig.add_trace(
    go.Scatter(
        x=np.arange(n_spins + 1),
        y=casino_wealth,
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 1)', width=2),
        name='Casino'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500,
    width=1300,
    showlegend=True,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

# Update axes
for i in range(1, 3):
    fig.update_xaxes(
        title='Number of Spins',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )
    fig.update_yaxes(
        title='Wealth ($)' if i == 1 else 'Casino Loss ($)',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(128,128,128,0.2)',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='rgba(128,128,128,0.5)',
        row=1, col=i
    )

fig.show()